# Load Packages

In [1]:
# import the different packages needed for the script to correctly load
import pandas as pd
import json
from datetime import datetime, date, timedelta
import hashlib
import requests
import os
import logging
import uuid
import binascii

# Parameters

## Filepaths / Read variables

In [2]:
# env.json hat following structure
# you may find your data under Web Service in https://sc3.omniture.com/p/suite/1.3/index.html?a=User.GetAccountInfo
'''
{
	"omniture": {
		"username": "USER NAME: COMPANY goes here",
		"secret": "SHARED SECRET goes here",
		"endpoint": "https://api.omniture.com/admin/1.4/rest/"
	}
}
'''
# Use ENV_empty.json from the Repo as template and save it as ENV.json
env_path = './ENV_VARIABLES/ENV.json'
env_variables = pd.read_json(env_path)

## Retrieving variables

In [3]:
# does are variables taken from the previous json file and are needed for the Auth with Adobe's APIs
USERNAME = env_variables.loc['username']['omniture']
SECRET = env_variables.loc['secret']['omniture']
DEFAULT_ENDPOINT = env_variables.loc['endpoint']['omniture']

## Are you behind a proxy?

In [4]:
# Add proxies if needed
http_proxy  = ""
https_proxy = ""
ftp_proxy   = ""

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }

# Functions

### Basic API functions (Authenticate, requests)

In [5]:
def authenticate(username, secret=None, endpoint=DEFAULT_ENDPOINT, prefix='', suffix=''):
    """ Authenticate to the Adobe API using WSSE """
    # if no secret is specified, we will assume that instead
    # we have received a dictionary with credentials (such as
    # from os.environ)
    if not secret:
        source = username
        # key_to_username = affix(prefix, 'USERNAME', suffix)
        # key_to_secret = affix(prefix, 'SECRET', suffix)
        username = USERNAME
        secret = SECRET

    return Account(username, secret, endpoint)

def affix(prefix=None, base=None, suffix=None, connector='_'):
    if prefix:
        prefix = prefix + connector
    else:
        prefix = ''

    if suffix:
        suffix = connector + suffix
    else:
        suffix = ''

    return prefix + base + suffix

class Account(object):
    """ A wrapper for the Adobe Analytics API. Allows you to query the reporting API """
    DEFAULT_ENDPOINT = 'https://api.omniture.com/admin/1.4/rest/'

    def __init__(self, username, secret, endpoint=DEFAULT_ENDPOINT, cache=False, cache_key=None):
        """Authentication to make requests."""
        self.log = logging.getLogger(__name__)
        self.log.info(datetime.now().strftime("%Y-%m-%d %I%p:%M:%S"))
        self.username = username
        self.secret = secret
        self.endpoint = endpoint

        self.suites = self.request('Company', 'GetReportSuites')['report_suites']

        # suites = [Suite(suite['site_title'], suite['rsid'], self) for suite in data]


    def request(self, api, method, query={}):
        """
        Make a request to the Adobe APIs.

        * api -- the class of APIs you would like to call (e.g. Report,
            ReportSuite, Company, etc.)
        * method -- the method you would like to call inside that class
            of api
        * query -- a python object representing the parameters you would
            like to pass to the API
        """
        self.log.info("Request: %s.%s  Parameters: %s", api, method, query)
        response = requests.post(
            self.endpoint,
            params={'method': api + '.' + method},
            data=json.dumps(query),
            headers=self._build_token(),
            proxies=proxyDict
            )
        self.log.debug("Response for %s.%s:%s", api, method, response.text)
        json_response = response.json()

        if type(json_response) == dict:
            self.log.debug("Error Code %s", json_response.get('error'))
            if json_response.get('error') == 'report_not_ready':
                return json_response
            elif json_response.get('error') != None:
                return json_response
            else:
                return json_response
        else:
            return json_response

    def _build_token(self):
        nonce = str(uuid.uuid4())
        base64nonce = binascii.b2a_base64(binascii.a2b_qp(nonce))
        created_date = datetime.utcnow().isoformat() + 'Z'
        sha = nonce + created_date + self.secret
        sha_object = hashlib.sha1(sha.encode())
        password_64 = binascii.b2a_base64(sha_object.digest())

        properties = {
            "Username": self.username,
            "PasswordDigest": password_64.decode().strip(),
            "Nonce": base64nonce.decode().strip(),
            "Created": created_date,
        }
        header = 'UsernameToken ' + self._serialize_header(properties)

        return {'X-WSSE': header}

    def getevars(self, rsid):
        """ Return the list of valid evars for the current report suite """
        return self.request('ReportSuite', 'GetEvars', query = rsid)


    def _serialize_header(self, properties):
        header = []
        for key, value in properties.items():
            header.append('{key}="{value}"'.format(key=key, value=value))
        return ', '.join(header)


# Authenticate Analytics API

In [6]:
# initialize connection to adobe
analytics = authenticate(os.environ)